## Variational Quantum Eigensolver

First, let us import necessary packages.

In [ ]:
import sys
import shutil
import tarfile
from google.colab import drive

drive.mount('/content/gdrive')
shutil.copy('/content/gdrive/MyDrive/qcintro.tar.gz', '.')
with tarfile.open('qcintro.tar.gz', 'r:gz') as tar:
    tar.extractall(path='/root/.local')

sys.path.append('/root/.local/lib/python3.10/site-packages')

!git clone -b branch-2024 https://github.com/UTokyo-ICEPP/qc-workbook-lecturenotes
!cp -r qc-workbook-lecturenotes/qc_workbook /root/.local/lib/python3.10/site-packages/

In [37]:
# Tested with python 3.10.14, qiskit 1.0.2, numpy 1.26.4, scipy 1.13.0
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter, ParameterVector
from qiskit.primitives import Estimator, BackendEstimator
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp
from qiskit_algorithms.optimizers import SPSA, COBYLA
from qiskit_aer import AerSimulator

### VQE Example 1
First, we try a simple example: optimize ansatz parameters using VQE so that the expectation value of an observable is minizized.

The observable is the product 'ZXY' of Pauli operators and the ansatz is the circuit with layers of $R_Y$ and $R_Z$ gates.

Finally, the minimized expectation values obtained using three different optimizers in VQE are compared with the exact value from exact diagonalization.

In [80]:
from qiskit_algorithms.minimum_eigensolvers import VQE, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import CG, GradientDescent
from qiskit_algorithms.gradients import ParamShiftEstimatorGradient

backend = AerSimulator()
estimator = BackendEstimator(backend)

In [ ]:
# Ansatzの定義
num_qubits = 3   # Number of qubits
num_layers = 2  # Number of layers

ansatz = QuantumCircuit(num_qubits)

# Vector of parameters (0)
theta = ParameterVector('θ')

# Function that a new element is added to theta and is returned
def new_theta():
    theta.resize(len(theta) + 1)
    return theta[-1]

for _ in range(num_layers):
    for iq in range(num_qubits):
        ansatz.ry(new_theta(), iq)

    for iq in range(num_qubits):
        ansatz.rz(new_theta(), iq)

    #for iq in range(num_qubits - 1):
    #    ansatz.cx(iq, iq + 1)

ansatz.draw('mpl')

In [77]:
# Observable
obs = SparsePauliOp('ZXY')

# Initial parameter values
rng = np.random.default_rng(999999)
init = rng.uniform(0., 2. * np.pi, size=len(theta))

# Instance to calcualte the gradient of observable with paramter shift rule using Estimator
grad = ParamShiftEstimatorGradient(estimator)

# VQE with conjugate gradient
optimizer_cg = CG(maxiter=200)
vqe_cg = VQE(estimator, ansatz, optimizer_cg, gradient=grad, initial_point=init)

# VQE with gradient descent
optimizer_gd = GradientDescent(maxiter=200)
vqe_gd = VQE(estimator, ansatz, optimizer_gd, gradient=grad, initial_point=init)

# VQE with COBYLA
optimizer_cobyla = COBYLA(maxiter=300)
vqe_cobyla = VQE(estimator, ansatz, optimizer_cobyla, initial_point=init)

# Exact diagonalization solver
ee = NumPyMinimumEigensolver()

In [78]:
#result_vqe_cg = vqe_cg.compute_minimum_eigenvalue(obs)
result_vqe_gd = vqe_gd.compute_minimum_eigenvalue(obs)
result_vqe_cobyla = vqe_cobyla.compute_minimum_eigenvalue(obs)
result_ee = ee.compute_minimum_eigenvalue(obs)

In [ ]:
print('Result:')
print(f'  Exact      = {result_ee.eigenvalue}')
print(f'  VQE(COBYLA) = {result_vqe_cobyla.optimal_value}')
#print(f'  VQE(CG)    = {result_vqe_cg.optimal_value}')
print(f'  VQE(GD)    = {result_vqe_gd.optimal_value}')

### VQE Example 2

Next task is to obtain the approximated lowest energy in some physics model, which is a prototypical problem for VQE. The physics model we consider here is the longitudinal-transverse field Ising model, one of the most popular benchmarks in condenced matter physics.

The ansatz circuit is composed of $R_Y$ gates and controlled-$Z$ gates.

In [ ]:
from qiskit_algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal

# VQE ansatz
num_qubits = 4

ansatz = TwoLocal(num_qubits, "ry", "cz", reps=3)  # Ry gates with trainable parameters and CZ for entanglement
optimizer = SLSQP(maxiter=1000)  # Classical optimizer
ansatz.decompose().draw('mpl')

The model has a parameter `alpha` that controls the mixture of longitudinal and transverse fieldd. The `alpha = 0` corresponds to pure transverse field, and `alpha = pi/2` to pure longitudinal field.


In [83]:
# Use Estimator
estimator = Estimator()

# VQE class in Qiskit
vqe = VQE(estimator, ansatz, optimizer)

# Hamiltonian of the longitudinal-transverse field Ising model
def get_hamiltonian(L, J, h, alpha=0):

    # Define the list composed of Hamiltonian terms as tuples
    # (1) Pauli string
    # (2) Index of qubits associated with Pauli string
    # (3) Coefficient of Pauli string
    ZZ_tuples = [("ZZ", [i, i + 1], -J) for i in range(0, L - 1)]
    Z_tuples = [("Z", [i], -h * np.sin(alpha)) for i in range(0, L)]
    X_tuples = [("X", [i], -h * np.cos(alpha)) for i in range(0, L)]

    # Construct Hamiltonian as SparsePauliOp object using `from_sparse_list`
    hamiltonian = SparsePauliOp.from_sparse_list([*ZZ_tuples, *Z_tuples, *X_tuples], num_qubits=L)
    return hamiltonian.simplify()

As an example, we attempt to obtain ground state energy at `alpha = pi/8`, where the longitudinal and transverse fields are mixed. The coupling strength between neighboring qubits is `J = 0.2`, and the coupling strength with external field is `h = 1.2`.

In [ ]:
# Parameter values
J = 0.2
h = 1.2
alpha = np.pi/8
H = get_hamiltonian(L=num_qubits, J=J, h=h, alpha=alpha)

# Calculate energy using VQE
result = vqe.compute_minimum_eigenvalue(H)
#print(result)
print(f'VQE energy value = {result.optimal_value:.5f}')

It is possible to calculate the true energy with exact diagonalization because the system size is small.

In [ ]:
# Exact diagonalization of the Hamiltonian
numpy_solver = NumPyMinimumEigensolver()
result = numpy_solver.compute_minimum_eigenvalue(operator=H)
ref_value = result.eigenvalue.real
print(f"Reference energy value = {ref_value:.5f}")